In [1]:
from db_connection import DB_Connection

In [2]:
import pandas as pd, numpy as np 
import matplotlib.pyplot as plt
from matplotlib.widgets import TextBox
import re, json
from wordcloud import WordCloud

In [3]:
db = DB_Connection()

Connection succesful.


In [44]:
df = pd.DataFrame(database, columns=['id','userid','username','text','date',
                            'link','media1','media2','media3', 'media4'])
df_clean = df[:]
df_clean = df_clean[df_clean.text!='']

In [57]:
# Cleaning the tweets
stop_words = json.load(open('./others/stop_words.json', encoding='utf8'))
def preprocess(tweet):
    # remove next lines
    tweet = re.sub('\n', ' ', tweet)
    # remove links
    tweet = re.sub('http\S+\s*', '', tweet)
    # remove between parentesis
    tweet = re.sub('\[.*\]','', tweet)
    # remove mentions
    tweet = re.sub("@\w+", "", tweet)
    # remove hashtags
    tweet = re.sub("#\w+", "", tweet)
    # alphanumeric and hashtags
    tweet = re.sub("[^a-zA-Z0-9ñáéíóúÁÉÍÓÚüÜ]", " ", tweet)
    # remove multiple spaces
    tweet = re.sub("\s+", " ", tweet)
    tweet = re.sub('^\s+', '', tweet)
    # lower first character
    try: tweet = tweet[0].lower() + tweet[1:]
    except: pass
    # remove stop words
    tweet = ' '.join([word for word in tweet.split(' ') 
                if not word.lower() in stop_words])

    return tweet

In [58]:
# df_processed = pd.DataFrame(columns=['id','text','clean','place','coords'])
df_processed = df_clean.loc[:, ['id', 'text']]
df_processed['clean'] = df_processed.text
df_processed['clean'] = df_processed.apply(lambda row: preprocess(row.clean), axis=1)
# for row in df:
#     index = len(df_processed)
#     print(row)
#     df_processed.iloc[index, 'id'] = row[0]
#     df_processed.iloc[index, 'text'] = row[3]
#     clean_twt = preprocess(row[3])
#     df_processed.iloc[index, 'clean'] = clean_twt
#     df_processed.iloc[index, 'place'] = get_geoloc(clean_twt)

# df_processed

In [62]:
df2 = df_processed[:]

In [63]:
df2['isAccident'] = np.where(df2.clean.str.contains('accidente|accidentaron|accidentó|choque|chocó|chocaron|choca|chocar|colisión|colisionaron|colisionó|colisiona|colisionado|vuelco|volcó|vuelca|atropello|atropellado'), 1, 0)

df2['isObstacle'] = np.where(df2.clean.str.contains('tranque|trancado|embotellamiento|desplazan autos|vistas tráfico|movimiento vehicular|huelga|motín|protesta|protestando|trabajos vía|trabajos|trabajos ruta|cierre|cerraron|cerrado|cierran|cierra|daño|dañó|detuvieron|detenido|detenida|obstáculo|obstaculizando|parado|paro|paño cerrado|tráfico paralizado|tráfico detenido|tráfico afectado|tráfico pesado|tráfico lento|desvío|área acordonada'), 1, 0)

df2['isDanger'] = np.where(df2.clean.str.contains('incendio|incendia|incendiando|incendiado|incendiaron|inundado|inundación|inundó'), 1, 0)

df2['isIncident'] = np.where(df2[['isAccident','isObstacle','isDanger']].sum(axis=1)>0, 1, 0)

In [14]:
df2['place'] = df2.apply(lambda row: get_place(row.clean) if row.isIncident==1 else '', axis=1)

# df2[['lat','long']] = df2.apply(lambda row: get_place(row.clean) if row.isIncident==1 else '', axis=1)